# Quantizing  Model



In [1]:
!pip install datasets -q
#!pip install transformers -q
!pip install bitsandbytes -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cud

### imports

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import BitsAndBytesConfig

### load data

In [3]:
data = load_dataset("KaushiGihan/phishing-site-url")
data

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/615 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/696k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/151k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/146k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14699 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3150 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3151 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 14699
    })
    validation: Dataset({
        features: ['text', 'labels'],
        num_rows: 3150
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 3151
    })
})

### load student model

In [4]:
device = torch.device('cuda')

In [5]:
# Load student model and tokenizer
model_id = "KaushiGihan/bert-text-classifier_KD_model"
tokenizer = AutoTokenizer.from_pretrained("KaushiGihan/bert-text-classifier")
model = AutoModelForSequenceClassification.from_pretrained(model_id).to(device)

tokenizer_config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/495 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/211M [00:00<?, ?B/s]

### tokenize data

In [6]:
# define text preprocessing
def preprocess_function(examples):
    return tokenizer(examples["text"], padding='max_length', truncation=True)

# tokenize all datasetse
tokenized_data = data.map(preprocess_function, batched=True)
tokenized_data.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

Map:   0%|          | 0/14699 [00:00<?, ? examples/s]

Map:   0%|          | 0/3150 [00:00<?, ? examples/s]

Map:   0%|          | 0/3151 [00:00<?, ? examples/s]

### Evaluation

In [7]:
# Function to evaluate model performance
def evaluate_model(model, dataloader, device):
    model.eval()  # Set model to evaluation mode
    all_preds = []
    all_labels = []

    # Disable gradient calculations
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # Forward pass to get logits
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # Get predictions
            preds = torch.argmax(logits, dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())

    # Calculate evaluation metrics
    accuracy = accuracy_score(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='binary')

    return accuracy, precision, recall, f1

### Evaluate model (pre-quantization)

In [8]:
# create data loader
validation_dataloader = DataLoader(tokenized_data['validation'], batch_size=128)

In [9]:
# Evaluate the student model
base_accuracy, base_precision, base_recall, base_f1 = evaluate_model(model, validation_dataloader, device)
print("Pre-quantization Performance")
print(f"Accuracy: {base_accuracy:.4f}, Precision: {base_precision:.4f}, Recall: {base_recall:.4f}, F1 Score: {base_f1:.4f}")

Pre-quantization Performance
Accuracy: 0.9283, Precision: 0.9250, Recall: 0.9332, F1 Score: 0.9291


### Quantize model

In [10]:
# load model in model as 4-bit
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype = torch.bfloat16,
    bnb_4bit_use_double_quant=True
)

model_nf4 = AutoModelForSequenceClassification.from_pretrained(model_id, device_map=device, quantization_config=nf4_config)

### Evaluate Model (post-quantization)

In [11]:
# Evaluate the student model
quantized_accuracy, quantized_precision, quantized_recall, quantized_f1 = evaluate_model(model_nf4, validation_dataloader, device)

print("Post-quantization Performance")
print(f"Accuracy: {quantized_accuracy:.4f}, Precision: {quantized_precision:.4f}, Recall: {quantized_recall:.4f}, F1 Score: {quantized_f1:.4f}")

Post-quantization Performance
Accuracy: 0.9263, Precision: 0.9169, Recall: 0.9388, F1 Score: 0.9277


### Push to Hub

In [12]:
from huggingface_hub import notebook_login

notebook_login()

In [13]:
model_nf4.push_to_hub("KaushiGihan/bert-text-classifier_KD_quantized_model")

model.safetensors:   0%|          | 0.00/62.7M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/KaushiGihan/bert-text-classifier_KD_quantized_model/commit/fc931e253639db9bf5d44ef5c928a21156b6557a', commit_message='Upload DistilBertForSequenceClassification', commit_description='', oid='fc931e253639db9bf5d44ef5c928a21156b6557a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/KaushiGihan/bert-text-classifier_KD_quantized_model', endpoint='https://huggingface.co', repo_type='model', repo_id='KaushiGihan/bert-text-classifier_KD_quantized_model'), pr_revision=None, pr_num=None)